### Charger les valeurs d'accuracy construites dans un précédent script (2018-02-16_clasiffier)

In [1]:
import numpy as np
import os

path = "MNIST_accuracy.npy"
if os.path.isfile(path):
    print('Loading accuracy...')
    accuracy =  np.load(path)
    print(accuracy)

Loading accuracy...
[[ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]
 [ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]
 [ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]
 ..., 
 [ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]
 [ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]
 [ 0.0974  0.0974  0.0974 ...,  0.0974  0.0974  0.0974]]


### Afficher un couple carte de certitude/image

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
from LogGabor import LogGabor

def vectorization(N_theta, N_orient, N_scale, N_phase, N_X, N_Y):
    phi = np.zeros((N_theta, N_orient, N_scale, N_phase, N_X*N_Y))
    parameterfile = 'https://raw.githubusercontent.com/bicv/LogGabor/master/default_param.py'
    lg = LogGabor(parameterfile)
    lg.set_size((N_X, N_Y))
    params= {'sf_0':.1, 'B_sf': lg.pe.B_sf, 'theta':np.pi* 5 / 7., 'B_theta': lg.pe.B_theta}
    phase = np.pi/4
    edge = lg.normalize(lg.invert(lg.loggabor(N_X/3, 3*N_Y/4, **params)*np.exp(-1j*phase)))
    
    for i_theta in range(N_theta):
        for i_orient in range(N_orient):
            for i_scale in range(N_scale):
                ecc =  .5**(N_scale - i_scale)
                r = np.sqrt(N_X**2+N_Y**2) / 2 * ecc # radius
                sf_0 = 0.5 * 0.03 / ecc
                x = N_X/2 + r * np.cos((i_orient+(i_scale % 2)*.5)*np.pi*2 / N_orient)
                y = N_Y/2 + r * np.sin((i_orient+(i_scale % 2)*.5)*np.pi*2 / N_orient)            
                for i_phase in range(N_phase):
                    params= {'sf_0':sf_0, 'B_sf': lg.pe.B_sf, 'theta':i_theta*np.pi/N_theta, 'B_theta': np.pi/N_theta/2}
                    phase = i_phase * np.pi/2
                    phi[i_theta, i_orient, i_scale, i_phase, :] = lg.normalize(lg.invert(lg.loggabor(x, y, **params)*np.exp(-1j*phase))).ravel()            
    return phi

N_theta, N_orient, N_scale, N_phase, N_X, N_Y = 6, 8, 5, 2, 128, 128
phi = vectorization(N_theta, N_orient, N_scale, N_phase, N_X, N_Y)
phi_vector = phi.reshape((N_theta*N_orient*N_scale*N_phase, N_X*N_Y))

def accuracy_128(i_offset, j_offset):
    N_pic = 128
    center = (128-55)//2
    
    #accuracy_128 = accuracy[0][0] * np.ones((N_pic,N_pic))
    accuracy_128 = 0.1 * np.ones((N_pic,N_pic))
    accuracy_128[(center+i_offset):(center+55+i_offset),(center+j_offset):(center+55+j_offset)] = accuracy
    
    accuracy_LP = phi_vector @ np.ravel(accuracy_128)
    
    return accuracy_LP

def mnist_128(data, i_offset, j_offset):
    N_pic = 128
    center = (128-28)//2
    
    data_128 = data.min() * np.ones((128,128))
    data_128[(center+i_offset):(center+28+i_offset),(center+j_offset):(center+28+j_offset)] = data

    data_LP = phi_vector @ np.ravel(data_128)
     
    return data_LP

def couples(data, i_offset, j_offset):
    v = mnist_128(data, i_offset, j_offset)
    a = accuracy_128(i_offset, j_offset)
    
    return (v,a)

def minmax(value, border):
    value = max(value, -border)
    value = min(value, border)
    return value

In [12]:
i_offset, j_offset = np.random.randn()*10, np.random.randn()*10 
couples(i_offset, j_offset)

(array([ -2.37780689e-03,   3.05003533e-03,  -9.24149246e-03,
         -1.72936450e-02,  -1.68931814e-01,  -1.62894933e-01,
          2.01524522e-01,  -1.13139074e+00,  -4.53293760e-01,
         -2.14000185e+00,  -2.26621179e-04,  -7.71483353e-04,
          1.53634136e-02,   8.14292051e-03,   1.40983082e-01,
         -1.11213371e-01,  -8.87080938e-02,   1.64309769e+00,
          2.92538732e+00,   2.70689820e+00,   1.66037391e-03,
          8.94968788e-04,  -3.05162786e-02,  -2.05435185e-02,
         -2.01986317e-01,   7.09279916e-02,   1.76053052e+00,
         -2.20183459e+00,  -1.03147402e+00,  -1.08170237e+01,
         -4.22747468e-03,   3.33215311e-03,   1.80120171e-01,
          1.67508344e-02,   2.22481804e-01,   2.19025454e-02,
         -2.24189139e+00,   8.08811169e+00,  -1.20058692e+01,
          7.61800823e+00,  -3.14599059e-02,  -1.45039359e-02,
          1.41304781e+00,   4.40094589e-01,   1.51874801e+00,
         -7.29858324e-01,  -8.73198470e+00,   2.26590232e+01,
        

### Construire un modèle simple de régression linéaire pour apprendre la correspondance v/a

In [17]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from LogGabor import LogGabor
import numpy as np
import time

data_sample = 100 #quantity of examples that'll be processed
v_size = 480      #size of the v vector, representing the processed image
epoch_nb = 1      #number of training epochs

data_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', 
                   train=True,    #def the dataset as training data 
                   download=True, #download if dataset not present on disk
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=data_sample, 
                   shuffle=True)

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self, data):
        data = F.relu(self.hidden(data))
        data = self.predict(data)
        return data
    
net = Net(n_feature=v_size, n_hidden=500, n_output=v_size)
optimizer = torch.optim.SGD(net.parameters(), lr=0.06)
loss_func = torch.nn.MSELoss()

t_start = time.time()
print('Starting training...')
for epoch in range(epoch_nb):
    for batch_idx, (data, label) in enumerate(data_loader):
        data_v, data_a = np.zeros((data_sample, 1, v_size)), np.zeros((data_sample, 1, v_size))
        for idx in range(data_sample):
            i_offset, j_offset = minmax(np.random.randn()*10, 20), minmax(np.random.randn()*10, 20)
            v, a = couples(data[idx,0,:], i_offset, j_offset)
            data_v[idx,0,:], data_a[idx,0,:] = v, a
            
        data_v, data_a = torch.FloatTensor(data_v), torch.FloatTensor(data_a)
        data_v, data_a = Variable(data_v), Variable(data_a)
        
        prediction = net(data_v)
        loss = loss_func(prediction, data_a)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if batch_idx % 50 == 0:
            print('Epoch {}: [{}/{}]; Loss: {:.6f}; Time: {:.2f} mn'.format(
                epoch, batch_idx*data_sample, len(data_loader.dataset), 
                loss.data.numpy(), (time.time()-t_start)/60))
print('...Training done')

Train epoch: 0, [0/60000], loss: [ 15.0872736], time: 1.87
Train epoch: 0, [5000/60000], loss: [ 3.91354418], time: 98.03
Train epoch: 0, [10000/60000], loss: [ 2.80551505], time: 193.77
Train epoch: 0, [15000/60000], loss: [ 2.57642341], time: 289.94
Train epoch: 0, [20000/60000], loss: [ 1.90989757], time: 386.15
Train epoch: 0, [25000/60000], loss: [ 2.00842166], time: 483.34
Train epoch: 0, [30000/60000], loss: [ 1.85034621], time: 580.50
Train epoch: 0, [35000/60000], loss: [ 1.78784788], time: 676.27
Train epoch: 0, [40000/60000], loss: [ 1.61416721], time: 772.70
Train epoch: 0, [45000/60000], loss: [ 1.39829302], time: 868.73
Train epoch: 0, [50000/60000], loss: [ 1.36424303], time: 965.05
Train epoch: 0, [55000/60000], loss: [ 1.45835841], time: 1060.96
Everything is ok!
